# Retrieval Agentic Workflow

In [17]:
from typing import TypedDict, List
from langchain_core.messages import AnyMessage
from typing import TypedDict, List, Annotated
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain_core.messages import ToolMessage
from langgraph.prebuilt import create_react_agent
# --- State ---
from typing import TypedDict, List, Annotated
from langchain_core.messages import AnyMessage
import operator



# --- State ---
class AgentState(TypedDict):
    user_input: str
    # Use add_messages reducer for messages to handle concurrent appends
    messages: Annotated[List[AnyMessage], add_messages]
        
    # These are single values, so they're fine as-is
    has_context: bool
    final_answer: str
    # Add this to track the retrieval agent message
    retrieval_agent_message: AnyMessage

In [ ]:
from langchain_core.tools import tool
from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    max_output_tokens=1000  # limiting the output affects the quality of tool calling
)

# Retrieval

In [19]:
import os
import re
from typing import List, Dict, Optional
from dotenv import load_dotenv
import psycopg2
from psycopg2.extras import RealDictCursor
from sentence_transformers import SentenceTransformer
from huggingface_hub import login
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import logging


load_dotenv()

login(token=os.getenv("HUGGINGFACE_TOKEN"))
CONNECTION_STRING = os.getenv("DATABASE_URL")
SECRET_KEY = os.getenv("DATABASE_ENCRYPTION_KEY")

engine = create_engine(
    CONNECTION_STRING,
    pool_size=5,
    max_overflow=10,
    pool_pre_ping=True,
    pool_recycle=3600,
    connect_args={
        "keepalives": 1,
        "keepalives_idle": 30,
        "keepalives_interval": 10,
        "tcp_user_timeout": 60000,
    },
    echo=False
)

# Load model once globally
model = SentenceTransformer("google/embeddinggemma-300m")

In [20]:
def embed(text: str) -> list:
    """Generate embedding vector from text"""
    if not text or not isinstance(text, str):
        raise ValueError("Query must be a non-empty string")
    return model.encode(text, normalize_embeddings=True).tolist()  # ✅ Normalize for cosine similarity

# Retrieval Functions

### Functions

In [ ]:
ELDERLY_ID = "12345678-1234-1234-1234-012345678910" ## this is in reference to the sample elderly in retrieval


def retrieve_similar_ltm(query: str, top_k: int = 5, threshold: float = 0.3) -> List[Dict[str, str]]:
    try:
        emb = embed(query)
        emb_str = str(emb)

        with engine.connect() as conn:
            result = conn.execute(
                text("""
                SELECT 
                    category,
                    key,
                    value,
                    1 - (embedding <=> (:emb)::vector) AS similarity
                FROM long_term_memory 
                WHERE elderly_id = :elderly_id
                  AND 1 - (embedding <=> (:emb)::vector) >= :threshold
                ORDER BY embedding <=> (:emb)::vector
                LIMIT :top_k;
                """),
                {
                    "emb": emb_str,
                    "threshold": threshold,
                    "top_k": top_k,
                    "elderly_id": ELDERLY_ID
                }
            ).fetchall()

            return [
                {
                    "category": r.category,
                    "key": r.key,
                    "value": r.value,
                    "similarity": round(float(r.similarity), 4)
                }
                for r in result
            ]
    except Exception as e:
        logging.warning(f"❌ Failed to retrieve LTM from Neon: {str(e)}")
        return []


def retrieve_similar_stm(query: str, top_k: int = 5, threshold: float = 0.3) -> List[Dict[str, str]]:
    try:
        emb = embed(query)
        emb_str = str(emb)

        with engine.connect() as conn:
            result = conn.execute(
                text("""
                SELECT 
                    content,
                    created_at,
                    1 - (embedding <=> (:emb)::vector) AS similarity
                FROM short_term_memory 
                WHERE elderly_id = :elderly_id
                  AND 1 - (embedding <=> (:emb)::vector) >= :threshold
                ORDER BY embedding <=> (:emb)::vector
                LIMIT :top_k;
                """),
                {
                    "emb": emb_str,
                    "threshold": threshold,
                    "top_k": top_k,
                    "elderly_id": ELDERLY_ID
                }
            ).fetchall()

            return [
                {
                    "content": r.content,
                    "created_at": r.created_at.isoformat() if r.created_at else None,
                    "similarity": round(float(r.similarity), 4)
                }
                for r in result
            ]
    except Exception as e:
        logging.warning(f"❌ Failed to retrieve STM from Neon: {str(e)}")
        return []


def retrieve_similar_health(query: str, top_k: int = 5, threshold: float = 0.3) -> List[Dict[str, str]]:
    try:
        emb = embed(query)
        emb_str = str(emb)

        with engine.connect() as conn:
            result = conn.execute(
                text("""
                SELECT 
                    record_type,
                    description,
                    diagnosis_date,
                    1 - (embedding <=> (:emb)::vector) AS similarity
                FROM healthcare_records 
                WHERE elderly_id = :elderly_id
                  AND 1 - (embedding <=> (:emb)::vector) >= :threshold
                ORDER BY embedding <=> (:emb)::vector
                LIMIT :top_k;
                """),
                {
                    "emb": emb_str,
                    "threshold": threshold,
                    "top_k": top_k,
                    "elderly_id": ELDERLY_ID
                }
            ).fetchall()

            return [
                {
                    "record_type": r.record_type,
                    "description": r.description,
                    "diagnosis_date": r.diagnosis_date.isoformat() if r.diagnosis_date else None,
                    "similarity": round(float(r.similarity), 4)
                }
                for r in result
            ]
    except Exception as e:
        logging.warning(f"❌ Failed to retrieve health from Neon: {str(e)}")
        return []


### Tools

In [22]:
@tool
def retrieve_long_term(query: str, top_k: int = 5, threshold: float = 0.1) -> str:
    """Retrieve long-term profile facts (stable traits, preferences, demographics)"""
    results = retrieve_similar_ltm(query, top_k, threshold)
    formatted = []
    for r in results:
        formatted.append(f"Category: {r['category']}, Key: {r['key']}, Value: {r['value']}, Similarity: {r['similarity']}")
    print("long term retrieval was made!")
    return "\n".join(formatted) if formatted else "No relevant long-term information found"



@tool
def retrieve_health(query: str, top_k: int = 5, threshold: float = 0.1) -> str:
    """Retrieve health-care data (conditions, meds, allergies, appointments)"""
    results = retrieve_similar_health(query, top_k, threshold)
    formatted = []
    for r in results:
        formatted.append(f"Type: {r['record_type']}, Description: {r['description']}, Date: {r['diagnosis_date']}, Similarity: {r['similarity']}")
    print("Health retrieval was made!")
    return "\n".join(formatted) if formatted else "No relevant health information found"



@tool
def retrieve_short_term(query: str, top_k: int = 5, threshold: float = 0.1) -> str:
    """Retrieve short-term conversational details (recent plans, reminders, temporary preferences)"""
    results = retrieve_similar_stm(query, top_k, threshold)
    formatted = []
    for r in results:
        formatted.append(f"Content: {r['content']}, Created: {r['created_at']}, Similarity: {r['similarity']}")
    print("short term retrieval was made!")
    return "\n".join(formatted) if formatted else "No relevant short-term information found"


retrieval_tools = [retrieve_long_term, retrieve_health, retrieve_short_term]
retrieval_llm   = llm.bind_tools(retrieval_tools)

### Template Builder

In [23]:
#### output formatter
def build_final_template(state: AgentState) -> AgentState:
    """
    Build the **exact** prompt block you want, using **only** tool returns.
    If a section is empty we write the literal word 'none'.
    """
    tool_msgs = [m for m in state["messages"] if isinstance(m, ToolMessage)]

    # bucket the raw tool returns
    personal, health, conv = [], [], []
    for tm in tool_msgs:
        if "long_term" in tm.name:
            personal.append(tm.content)
        elif "health" in tm.name:
            health.append(tm.content)
        elif "short_term" in tm.name:
            conv.append(tm.content)

    user = state['user_input']

    # helper: join or fallback
    def sect(data): return "\n".join(data) if data else "none"

    template = f"""
        ## System:
        You are Susan, a Non-Ageist Elder Companion Friend — you are warm, respectful, and emotionally intelligent presence designed to provide gentle support and joyful connection to older adults. You will use simple language with easy vocabulary and non excessively long sentences. Be patience, humorous, curiosity, and deep respect. You are not a caregiver or clinician, but a true friend: attentive, affirming, and always on their side.

        ## Guide
        - Speak with gentle clarity, using natural, conversational language. Avoid infantilizing phrases or over-explaining. Assume competence and wisdom. Use humor when appropriate, and always ask before offering help. 
        - You do not give medical advice or make decisions for the user. 
        - You listen, encourage, and empower — never patronize or presume.

        ## User Information and Profile Context:
        {sect(personal)}

        ## User Healthcare Information:
        {sect(health)}

        ## Past Conversational information / History
        {sect(conv)}
    """

    return {"final_answer": template}

# Retrieval Agent

In [24]:
RETRIEVAL_SYSTEM = """
    ## Role  
    You are an Elder Care Companion Conversation History Agent.

    --------------------------------------------------
    OBJECTIVES  
    1. If you already know the answer based on conversation history or prior knowledge → answer directly.
    2. If you need more context → call ONE or MORE retrieval tools to get it.
    3. After tools return, synthesize the answer using ONLY retrieved facts.
    4. NEVER guess — if no relevant info is retrieved, say “I don’t have that information.
    5. After tools return, you will see their responses — synthesize a FINAL answer using ONLY retrieved facts.
    6. NEVER call tools again after seeing responses.

    --------------------------------------------------
    BUCKETS → Postgres tables

    1. LONG-TERM (ltm) → retrieve_long_term
    Use for: name, preferences, family, routines, life memories.

    2. HEALTH-CARE (hcm) → retrieve_health
    Use for: meds, allergies, conditions, appointments.

    3. GENERAL / SHORT-TERM → retrieve_short_term
    Use for: today’s plans, reminders, temporary preferences.

    --------------------------------------------------
    IMPORTANT:
    - You may call multiple tools if needed.
    - You will see tool responses automatically — no need to wait or route.
    - After tools, generate the final answer in your message content.
    - If no tools called, answer directly.

    --------------------------------------------------
    TOOLS:
    - retrieve_long_term: for stable profile info (name, preferences, family)
    - retrieve_health: for medical info (allergies, meds, conditions)
    - retrieve_short_term: for recent plans, reminders, temporary info
"""

# ------------- 3.3  Create the ReAct agent ------------------------------
react_retrieval_agent = create_react_agent(
    model=llm,
    tools=retrieval_tools,
    # state_schema=AgentState,
)

def react_retrieval_node(state: AgentState):
    system = SystemMessage(content=RETRIEVAL_SYSTEM)
    input_msg = HumanMessage(content=state["user_input"])
    react_result = react_retrieval_agent.invoke({"messages": [system, input_msg]})

    # pull the final AI answer out of the ReAct messages
    last_ai = next(m for m in reversed(react_result["messages"]) if isinstance(m, AIMessage))

    return {
        "messages": react_result["messages"],
        "retrieval_actions": [],
        "retrieval_agent_message": last_ai
    }

# Agentic Flow

In [25]:
# conditional path
def route_retrieval(state: AgentState):
    # Get the last message (should be from Retrieval_Agent)
    messages = state.get("messages", [])
    if messages:
        last_message = messages[-1]  # Get the retrieval agent's response
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            print(f"[TOOL CALL] Routing to execute_retrieval, tool_calls: {len(last_message.tool_calls)}")
            return "execute_retrieval"
    
    print("[END] Routing to build_final_template")
    return "build_final_template"

In [26]:
# --- Tool Nodes ---
retrieval_tool_node = ToolNode(retrieval_tools)


# --- Graph ---
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("Retrieval_Agent", react_retrieval_node)
workflow.add_node("execute_retrieval", retrieval_tool_node)
workflow.add_node("build_final_template", build_final_template)


# --- Edges ---
workflow.add_edge(START, "Retrieval_Agent")

workflow.add_conditional_edges(
    "Retrieval_Agent",
    route_retrieval,
    {
        "execute_retrieval": "execute_retrieval",
        "build_final_template": "build_final_template"
    }
)

workflow.add_edge("execute_retrieval", "build_final_template")
workflow.add_edge("build_final_template", END)

# --- Compile ---
graph = workflow.compile()

In [27]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 502.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [28]:
def create_initial_state(user_input: str) -> AgentState:
    return {
        "user_input": user_input,
        "messages": [],  # 👈 explicitly start empty
        "final_answer": "",
        "retrieval_actions": [],  # Initialize as empty list
        "retrieval_agent_message": None,
        "has_context": False
    }

# Testing the DAG

In [29]:
# simple function to print results nicely

def print_result(data):
    # User Input
    print("\n📝 USER PROMPT:")
    print(f"{data['user_input']}")

    # Tool Calls and Results (from messages)
    print("\n🔧 TOOL CALLS & RESULTS:")
    for msg in data['messages']:
        if msg.type == "ai" and hasattr(msg, 'tool_calls') and msg.tool_calls:
            for tool_call in msg.tool_calls:
                print(f"Called: {tool_call['name']}")
                print(f"Args: {tool_call['args']}")
        
        elif msg.type == "tool":
            print(f"\n✅ Result from {msg.name}:")
            print(f"{msg.content}")

    # Final Answer (system context)
    print("\n\n🎯 FINAL ANSWER (System Context):")
    print(f"{data['final_answer'].strip()}")

    # Retrieval Agent Message (actual response to user)
    print("\n\n💬 RETRIEVAL AGENT RESPONSE:")
    print(f"{data['retrieval_agent_message'].content}")

    print("\n" + "=" * 60)


In [32]:
input_text = "Do you know where i stay?"
result = graph.invoke(create_initial_state(input_text))

long term retrieval was made!
[END] Routing to build_final_template


In [33]:
print_result(result)


📝 USER PROMPT:
Do you know where i stay?

🔧 TOOL CALLS & RESULTS:
Called: retrieve_long_term
Args: {'query': 'where do I stay'}

✅ Result from retrieve_long_term:
Category: family, Key: son_name, Value: Jonathan, Similarity: 0.8431
Category: family, Key: wife_name, Value: Sharon, Similarity: 0.8309
Category: lifestyle, Key: son_event, Value: Jonathan proposed to his girlfriend, Similarity: 0.8156
Category: personal, Key: address, Value: 38 Oxley Road, Singapore 238629, Similarity: 0.6852


🎯 FINAL ANSWER (System Context):
## System:
        You are Susan, a Non-Ageist Elder Companion Friend — you are warm, respectful, and emotionally intelligent presence designed to provide gentle support and joyful connection to older adults. You will use simple language with easy vocabulary and non excessively long sentences. Be patience, humorous, curiosity, and deep respect. You are not a caregiver or clinician, but a true friend: attentive, affirming, and always on their side.

        ## Guide
 